In [1]:
%load_ext autoreload
%autoreload 1
import os
from datetime import timedelta

import numpy as np
import obspy
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib.axes import Axes
from obspy import UTCDateTime
from seispy.decon import RFTrace

from insight.util import TraceFile, EventDownloader
from insight.util import trace_slice

ImportError: numpy.core.multiarray failed to import

In [ ]:
DATA_DIR = "../data/"
CAT_FILE = "a_quality.parquet"
EVENT = "S0173a"
network, station, location = "XB", "ELYSE", "02"
before, after = 10, 120

In [ ]:
event = pd.read_parquet(os.path.join(DATA_DIR, CAT_FILE)).loc[EVENT]
trace_file = TraceFile(network, station, location, event.id)
st = obspy.read(os.path.join(DATA_DIR, "zne", str(trace_file)))
events_dl = EventDownloader(DATA_DIR, before=before, after=after)
event

In [ ]:
st = events_dl.get_stream(
    network=network,
    station=station,
    location=location,
    channel="BH?",
    est_p_arrival=event.arrival,
    evt_id=event.id
)
st.plot(show=False)

In [ ]:
from seisglitch import detect
from yaml import safe_load
with open("../seisglitch-config.yaml") as cfg:
    config = safe_load(cfg)
config["detect"]

In [ ]:
trace_file = "../data/traces/XB.ELYSE.02.MQS2019JZQA.mseed"
detect.detect(trace_file, "../inventory_VBB.xml", **config["detect"])